In [1]:
from ultralytics import YOLO
import torch

model = YOLO("dyolov10n.yaml").cuda()
input = torch.ones(16, 640, 640, 3).cuda()

/home/dblab/anaconda3/envs/dconv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model(input.permute(0, 3, 1, 2))

0: 640x640 (no detections), 1.1ms
1: 640x640 (no detections), 1.1ms
2: 640x640 (no detections), 1.1ms
3: 640x640 (no detections), 1.1ms
4: 640x640 (no detections), 1.1ms
5: 640x640 (no detections), 1.1ms
6: 640x640 (no detections), 1.1ms
7: 640x640 (no detections), 1.1ms
8: 640x640 (no detections), 1.1ms
9: 640x640 (no detections), 1.1ms
10: 640x640 (no detections), 1.1ms
11: 640x640 (no detections), 1.1ms
12: 640x640 (no detections), 1.1ms
13: 640x640 (no detections), 1.1ms
14: 640x640 (no detections), 1.1ms
15: 640x640 (no detections), 1.1ms
Speed: 0.0ms preprocess, 1.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: '10', 11: '11', 12: '12', 13: '13', 14: '14', 15: '15', 16: '16', 17: '17', 18: '18', 19: '19', 20: '20', 21: '21', 22: '22', 23: '23', 24: '24', 25: '25', 26: '26', 27: '27', 28: '28', 29: '29', 30: '30', 31: '31', 32: '32', 33: '33', 34: '34', 35: '35', 36: '36', 37: '37', 38: '38', 39: '39', 40: '40', 41: '41', 42: '42', 43: '43', 44: '44', 45: '45', 46: '46', 47: '47', 48: '48', 49: '49', 50: '50', 51: '51', 52: '52', 53: '53', 54: '54', 55: '55', 56: '56', 57: '57', 58: '58', 59: '59', 60: '60', 61: '61', 62: '62', 63: '63', 64: '64', 65: '65', 66: '66', 67: '67', 68: '68', 69: '69', 70: '70', 71: '71', 72: '72', 73: '73', 74: '74', 75: '75', 76: '76', 77: '77', 78: '78', 79: '79'}
 obb: None
 orig_img: array([[[255, 255, 255],
         [255, 255, 255]

In [3]:
train_results = model.train(
    data = 'coco8.yaml',
    epochs=100,
    imgsz=640,
    device='0'
)

New https://pypi.org/project/ultralytics/8.3.0 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.100 🚀 Python-3.8.0 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce RTX 4090, 24214MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=dyolov10n.yaml, data=coco8.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, cl

100%|██████████| 433k/433k [00:00<00:00, 948kB/s] 
Unzipping /home/dblab/ML/dconv/datasets/coco8.zip to /home/dblab/ML/dconv/datasets/coco8...: 100%|██████████| 25/25 [00:00<00:00, 11675.49file/s]

Dataset download success ✅ (1.8s), saved to /home/dblab/ML/dconv/datasets


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1      9856  ultralytics.nn.modules.block.SCDown          [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7         

dYOLOv10n summary: 392 layers, 2,575,296 parameters, 2,575,280 gradients

Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


100%|██████████| 6.25M/6.25M [00:01<00:00, 4.71MB/s]


...

                   all          4         17          0          0          0          0



100 epochs completed in 0.006 hours.
Optimizer stripped from /home/dblab/ML/dconv/ultralytics/runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from /home/dblab/ML/dconv/ultralytics/runs/detect/train/weights/best.pt, 5.5MB

Validating /home/dblab/ML/dconv/ultralytics/runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.100 🚀 Python-3.8.0 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce RTX 4090, 24214MiB)
dYOLOv10n summary (fused): 294 layers, 2,562,640 parameters, 0 gradients


/home/dblab/ML/dconv/ultralytics/ultralytics/nn/modules/head.py:154: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return torch.cat([boxes[i, index // nc], scores[..., None], (index % nc)[..., None].float()], dim=-1)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 95.11it/s]

                   all          4         17          0          0          0          0


Speed: 0.1ms preprocess, 2.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /home/dblab/ML/dconv/ultralytics/runs/detect/train


In [4]:
conv.weight.data.shape

torch.Size([64, 64, 1, 1])

In [51]:
from torch.utils.benchmark import Timer

timer = Timer(
    stmt = 'output = layer(data)',
    setup='data = input',
    globals={'input': input, 'layer': linear}
)
print(timer.blocked_autorange(min_run_time=1))

output = layer(data)
setup: data = input
  Median: 811.17 us
  IQR:    7.21 us (809.07 to 816.28)
  13 measurements, 100 runs per measurement, 1 thread


In [52]:
from torch.utils.benchmark import Timer

timer = Timer(
    stmt = 'output = layer(data)',
    setup='data = input',
    globals={'input': input.permute(0,3,1,2), 'layer': conv}
)
print(timer.blocked_autorange(min_run_time=1))

output = layer(data)
setup: data = input
  Median: 1.64 ms
  IQR:    0.01 ms (1.64 to 1.65)
  7 measurements, 100 runs per measurement, 1 thread


In [54]:
N, H, W, C = input.shape
l = linear(input.view(N,H,W,C)).view(N,H,W,C)
c = conv(input.permute(0,3,1,2)).permute(0,2,3,1)

In [5]:
a = {'a':1, 'b':2}